This notebook describes the building load model in the powerflow module.

# Model

Buildings can be simulated using a reduced-order model using a discrete transfer function based on the thermal properties of the building.  The transfer function can be derived for a two-variable model using the indoor air temperature $T_A$ and building mass temperature $T_M$ as they respond to changes in outdoor air temperature $T_O$ and changes in heat gains to the air $Q_A$ and mass $Q_M$, i.e.,
$$
    C_A \dot T_A = U_A ( T_O - T_A ) + U_I ( T_M - T_A ) + Q_A \qquad (1)
$$
$$
    C_M \dot T_M = U_I (T_A - T_M ) + U_M ( T_O - T_M ) + Q_M \qquad (2)
$$
where
* $C_A$ is the heat capacity of the air volume;
* $C_M$ is the heat capacity of the building mass;
* $U_A$ is the conductance between the indoor to outdoor air;
* $U_I$ is the conductance between the indoor air and the building mass,; and
* $U_M$ is the conductance between the building mass and the outdoor air.

The heat gains to the air come from the following sources:
* $Q_E E$, the internal enduse electric loads, where $Q_E$ is installed capacity of electric end-uses and $E$ is the fraction of electric end-uses active;
* $Q_G G$, the internal enduse gas loads, where $Q_G$ is the heat retained in the building for installed capacity of gas enduses and $G$ is the fraction of gas end-uses active;
* $Q_O H$, the occupancy heat gains where $Q_O$ is the heat gain for a single person, and $H$ is the number of people present;
* $Q_V H$, the ventilation heat gains, where $Q_V$ is the heat gain for fresh air per person; and
* $M Q_H$, the HVAC heat gains, where $M$ is the system mode, and $Q_H$ is the nameplate system capacity.
Thus
$$
    Q_A = E Q_E + G Q_G + H (Q_O+Q_V) + M Q_H
$$

The heat gains to the mass from the solar gains to building envelope and mass through windows, such that $Q_M = S_A Q_S$, where $S_A$ is the solar exposure area of the building mass, and $Q_S$ is the total insolation.

The HVAC system mode is included as a state variable such that the mode $M$ represents the state of the HVAC system given the indoor air temperature setpoint $T_S$ deviation from $T_A$ with a proportional gain $K$, where $M>0$ for heating, and $M<0$ for cooling. 

The state space model is thus
$$
    \left[ \begin{matrix} \dot T_A \\ \dot T_M \\ \dot M \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        -{U_A+U_I \over C_A} & {U_I \over C_A} & {Q_H \over C_A}
    \\
        {U_I \over C_M} & -{U_M+U_I \over C_M} & 0
    \\
        K & 0 & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_A \\ T_M \\ M \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        { U_A \over C_A } & {Q_E \over C_A} & {Q_G \over C_A} & {Q_O+Q_V \over C_A} & 0 & 0
    \\
        { U_M \over C_M } & 0 & 0 & 0 & {S_A \over C_M} & 0
    \\
        0 & 0 & 0 & 0 & 0 & -K
    \end{matrix} \right]
    \left[ \begin{matrix} T_O \\ E \\ G \\ H \\ Q_S \\ T_S \end{matrix} \right]
    \qquad(3)
$$

In [221]:
from numpy import *
from numpy.linalg import *

# thermal parameters
UA = 300 # conductance from interior air to outdoor air (W/K)
CA = 2e6 # heat capacity of indoor air volume (J/K)
UI = 6000 # conductance from building mass to indoor air (W/K)
CM = 8e6 # heat capacity of building mass (J/K)
UM = 600 # conductance of building mass to outdoor air (W/K)

# design parameters
TH = -20 # heating design temperature (degC)
TC = 40 # cooling design temperature (degC)
DF = 0.5 # system overdesign factor (pu)
QH = None # system capacity, or None to autosize (W)
QE = 10000 # nominal electric enduse installed capacity (W/pu)
QG = 1000 # nominal gas enduse installed capacity heat to space W/pu
QO = 1200 # nominal heat gain from occupants (W/pu)
QV = 400 # nominal ventilation heat gain for occupants (W/pu)
SA = 10.0 # solar exposure area (m^2)

# control parameters
K = 1.0 # HVAC control gain w.r.t temperature

# inputs
TO = -20 # outdoor air temperature (degC)
EU = 0.1 # fraction of electric end-use (pu nominal)
NG = 0.1 # fraction of gas end-uses (pu nominal)
NH = 1.0 # occupancy (pu nominal)
QS = 1000 # insolation (W/m^2)
TS = 20 # indoor air temperature setpoint (degC)

# System Sizing

If the system size is not given, then $Q_H$ must be calculated for the design conditions.  The heating system capacity is found by solving Eq. (3) at steady-state for $Q_H$ given the $T_O=T_H$, $T_A=T_S$, $E=G=H=Q_S=0$, and $M=D_F$, the system over-design factor. This gives us the system
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {D_F \over C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_H \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} \\
        {U_M \over C_M} & {U_I \over C_M}
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \end{matrix} \right]
$$

The same method is used for cooling but with all internal gains on and maximum solar gain.
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {D_F \over C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_H \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} & 0 \\
        {U_M \over C_M} & {U_I \over C_M} & { QE+QG+QO+QV+1300S_A \over CA }
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \\ 1 \end{matrix} \right]
$$

In [222]:
# system autosize
if QH == None:
    
    # heating
    Ah = array([
        [UI/CA,DF/CA],
        [-(UM+UI)/CM,0.0]])
    Bh = array([
        [UA/CA,-(UA+UI)/CA],
        [UM/CM,UI/CM]])
    uh = array([[TH],[TS]])
    bh = -Bh@uh
    xh = solve(Ah,bh)
    
    # cooling
    Ac = array([
        [UI/CA,DF/CA],
        [-(UM+UI)/CM,0]])
    print(Ac)
    Bc = array([
        [UA/CA,-(UA+UI)/CA,0.0],
        [UM/CM,UI/CM,(QE+QG+QO+QV+1300*SA)/CM]])
    print(Bc)
    uc = array([[TC],[TS],[1.0]])
    print(uc)
    bc = -Bc@uc
    print(bc)
    xc = solve(Ac,bc)
    print(xc)
    
    QH = max(xh[1][0],-xc[1][0])
    print("Autosize QH:",QH.round(0),"W")
    

[[  0.0030   0.0000]
 [ -0.0008   0.0000]]
[[  0.0001  -0.0032   0.0000]
 [  0.0001   0.0008   0.0032]]
[[ 40.0000]
 [ 20.0000]
 [  1.0000]]
[[  0.0570]
 [ -0.0212]]
[[ 25.6970]
 [-80363.6364]]
Autosize QH: 80364.0 W


In [225]:
# system model
A = array([
     [-(UA+UI)/CA, UI/CA, QH/CA],
     [UI/CM, -(UM+UI)/CM, 0],
     [K, 0, 0]
    ])
B = array([
     [UA/CA, QE/CA, QG/CA, (QO+QV)/CA, 0, 0],
     [UM/CM, 0, 0, 0, SA/CM, 0],
     [0, 0, 0, 0, 0, -K]
    ])
print(A)
print(B)

[[ -0.0032   0.0030   0.0402]
 [  0.0008  -0.0008   0.0000]
 [  1.0000   0.0000   0.0000]]
[[  0.0001   0.0050   0.0005   0.0008   0.0000   0.0000]
 [  0.0001   0.0000   0.0000   0.0000   0.0000   0.0000]
 [  0.0000   0.0000   0.0000   0.0000   0.0000  -1.0000]]


In [224]:
# check system capacity
bh = -B@array([[TH],[0],[0],[0],[0],[TS]])
xh = solve(A,bh)
if xh.round(4)[2] > DF: 
    print(f'insufficient heating capacity at T={TH:+.0f}: M = {xh[2][0]*100:+.1f}% > +DF = {DF*100:+.1f}%')
print(bh)
print(xh)
-B@array([[TC],[1],[1],[1],[1],[TS]])
xc = solve(A,bc)
if xc.round(4)[2] < -DF: 
    print(f'insufficient cooling capacity at T={TC:+.0f}: M = {xc[2][0]*100:+.1f}% < -DF = {-DF*100:+.1f}%')
print(bc)
print(xc)

[[  0.0030]
 [  0.0015]
 [ 20.0000]]
[[ 20.0000]
 [ 16.3636]
 [  0.4208]]


ValueError: solve: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m,n)->(m,n) (size 2 is different from 3)

In [212]:
u = array([[TO],[EU],[NG],[NH],[QS],[TS]])
x = solve(A,-B@u)
TA = x[0][0]
TM = x[1][0]
M = x[2][0]
set_printoptions(formatter={'float_kind':"{:8.4f}".format})
print(f"TA: {TA:+5.1f} degC")
print(f"TM: {TM:+5.1f} degC")
print(f"M:  {M*100:+5.1f} %")


TA: +20.0 degC
TM: +17.9 degC
M:  +27.4 %


The real and reactive ZIP components, $\mathbf P$ and $\mathbf Q$, respectively, are obtained from the enduse loads $Q_E$ and the state of the HVAC system $M$, including the system efficiency $\eta$, and the real and reactive power load compositions:
$$
    \left[ \begin{matrix} P_Z \\ P_I \\ P_P \\ Q_Z \\ Q_I \\ Q_P \end{matrix} \right]
    =
    \left[ \begin{matrix}
        0 & 0 & P_{Z_M} 
    \\
        0 & 0 & 0
    \\
        0 & 0 & P_{P_M} 
    \\
        0 & 0 & 0
    \\
        0 & 0 & 0 
    \\
        0 & 0 & Q_{P_M}
    \end{matrix} \right] 
    \left[ \begin{matrix} T_A \\ T_M \\ M \end{matrix} \right]
    +
    \left[ \begin{matrix}
        0 & P_{Z_E} & 0 & 0 & 0 & 0 
    \\
        0 & P_{I_E} & 0 & 0 & 0 & 0 
    \\
        0 & P_{P_E} & 0 & P_{P_H} & 0 & 0 
    \\
        0 & Q_{Z_E} & 0 & 0 & 0 & 0 
    \\
        0 & Q_{I_E} & 0 & 0 & 0 & 0 
    \\
        0 & Q_{P_E} & 0 & Q_{P_H} & 0 & 0 
    \end{matrix} \right] 
    \left[ \begin{matrix} T_O \\ E \\ G \\ H \\ Q_S \\ T_S \end{matrix} \right]
$$

In [9]:
# outputs
PZM = 0 # constant impedance HVAC real power (W)
PPM = 2*QH # constant power HVAC real power (W)
QPM = 0.2*QH # constant power HVAC reactive power (VAr)
PZE = 500 # constant impedance end-use real power (W)
PIE = 0 # constant current end-use real power (W)
PPE = 500 # constant power end-use real power (W)
QZE = 50 # constant impedance end-use reactive power (VAr)
QIE = 0 # constant current end-use reactive power (VAr)
QPE = 50 # constant power end-use reactive power (VAr)
PPH = 300 # constant power ventilation real power (W/person)
QPH = 30 # constant power ventilation reactive power (VAr/person)

C = array([
     [0, 0, PZM],
     [0, 0, 0],
     [0, 0, PPM],
     [0, 0, 0],
     [0, 0, 0],
     [0, 0, QPM]
    ])
D = array ([
     [0, PZE, 0, 0, 0, 0],
     [0, PIE, 0, 0, 0, 0],
     [0, PPE, 0, 0, PPH, 0],
     [0, QZE, 0, 0, 0, 0],
     [0, QIE, 0, 0, 0, 0],
     [0, QPE, 0, 0, QPH, 0],
    ])

In [7]:
def quad(augmented=True,cooling=-1):
    if augmented:
        A = array([
             [(UA+UI)/CA, -UI/CA, -QH/CA],
             [-UI/CM, (UM+UI)/CM, 0],
             [K, 0, 0]
            ])
        B = array([
             [UA/CA, QE/CA, QG/CA, (QO+QV)/CA, 0, 0],
             [UM/CM, 0, 0, 0, SA/CM, 0],
             [0, 0, 0, 0, 0, -K]
            ])
        C = array([
             [0, 0, cooling*PZM if cooling else PZM],
             [0, 0, 0],
             [0, 0, cooling*PPM if cooling else PPM],
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, cooling*QPM if cooling else QPM]
            ])
        D = array ([
             [0, PZE, 0, 0, 0, 0],
             [0, PIE, 0, 0, 0, 0],
             [0, PPE, 0, 0, PPH, 0],
             [0, QZE, 0, 0, 0, 0],
             [0, QIE, 0, 0, 0, 0],
             [0, QPE, 0, 0, QPH, 0],
            ])
    else:
        A = array([
             [(UA+UI)/CA, -UI/CA],
             [-UI/CM, (UM+UI)/CM]
            ])
        B = array([
             [UA/CA, QE/CA, QG/CA, (QO+QV)/CA, 0],
             [UM/CM, 0, 0, 0, SA/CM]
            ])
        C = array([
             [0, 0],
             [0, 0],
             [0, 0],
             [0, 0],
             [0, 0],
             [0, 0]
            ])
        D = array ([
             [0, PZE, 0, 0, 0],
             [0, PIE, 0, 0, 0],
             [0, PPE, 0, 0, PPH],
             [0, QZE, 0, 0, 0],
             [0, QIE, 0, 0, 0],
             [0, QPE, 0, 0, QPH],
            ])
    return A,B,C,D

def steadystate(TO=TO,hvac='auto',with_gains=True,cooling=0,astype=array):
    if hvac == 'auto':
        A,B,C,D = quad(True,cooling)
        u = array([[TO],
                   [EU if with_gains else 0],
                   [NG if with_gains else 0],
                   [NH if with_gains else 0],
                   [QS if with_gains else 0],
                   [TS]])
        xt = ['TA','TM','M']
    elif hvac == 'off':
        A,B,C,D = quad(False)
        u = array([[TO],
                   [EU if with_gains else 0],
                   [NG if with_gains else 0],
                   [NH if with_gains else 0],
                   [QS if with_gains else 0]])
        xt = ['TA','TM']
    else:
        raise Exception(f"hvac='{hvac}' is invalid")
    x = solve(A,B@u).round(2)
    if hvac=='auto':
        M = x[2,0]
        if M<-0.1 and not cooling:
            raise Exception(f"cooling mode not set when needed (TO={TO}, M={M})")
        if M>0.1 and cooling:
            raise Exception(f"cooling mode set when not needed (TO={TO}, M={M})")
    y = (C@x+D@u).round(1)
    yt = ['PZ','PI','PP','QZ','QI','QP']
    if type(astype) == str:
        astype = eval(astype)
    if astype == array:
        return x,y
    elif astype == list:
        return list(x.T[0]),list(y.T[0])
    elif astype == dict:
        return {'x':dict(zip(xt,x.T[0])),'y':dict(zip(yt,y.T[0]))}
    else:
        raise Exception(f"astype={astype.__name__} is invalid")

print("Steady state results:")
print(f"Heating design (TH={TH:+3.0f}): x = {steadystate(TH,hvac='auto',astype=dict,with_gains=False)['x']}")
print(f"Cooling design (TC={TC:+3.0f}): x = {steadystate(TC,hvac='auto',astype=dict,cooling=-1)['x']}")
for T in arange(-10,50,10):
    print(f"Outdoor T={T:+3.0f} w/o HVAC: x = {steadystate(T,hvac='off',astype=dict)['x']}")
    print(f"Outdoor T={T:+3.0f} w HVAC: x = {steadystate(T,hvac='auto',astype=dict,cooling=(-1 if T>20 else 0))['x']}")
print(f"Heating design (TH={TH:+3.0f}): y = {steadystate(TH,hvac='auto',astype=dict,with_gains=False)['y']}")
print(f"Cooling design (TC={TC:+3.0f}): y = {steadystate(TC,hvac='auto',astype=dict,cooling=-1)['y']}")

Steady state results:
Heating design (TH=-20): x = {'TA': -10.0, 'TM': -10.91, 'M': 0.17}
Cooling design (TC=+40): x = {'TA': -10.0, 'TM': -2.42, 'M': -1.25}
Outdoor T=-10 w/o HVAC: x = {'TA': 14.13, 'TM': 14.97}


Exception: cooling mode not set when needed (TO=-10, M=-0.41)

The canonical state-space model is for this system is
$$
    \dot x = A~x + B~u
$$
$$
    \dot y = C~x + D~u
$$
where
* $ x = \left[ \begin{matrix} T_A \\ T_M \end{matrix} \right] $,
* $ A = \left[ \begin{matrix}
        {U_A+U_I \over C_A} & -{U_I \over C_A}
    \\
        -{U_A \over C_M} & {U_M+U_I \over C_M}
    \end{matrix} \right] $,
* $ B = \left[ \begin{matrix}
        -{U_A \over C_A} & {1 \over C_A} & 0
    \\
        -{U_M \over C_A} & 0 & {1 \over C_M}
    \end{matrix} \right] $,
* $ u = \left[ \begin{matrix} T_O \\ Q_A \\ Q_M \end{matrix} \right] $,
* $ y = \left[ \begin{matrix} P \\ Q \end{matrix} \right] $,
* $ C = \left[ \begin{matrix} \end{matrix} \right] $,
* $ D = \left[ \begin{matrix}
        -{U_A \over C_A} & {1 \over C_A} & 0
    \\
        -{U_M \over C_A} & 0 & {1 \over C_M}
    \end{matrix} \right] $,

The equilibrium condition $\bar x$ is computed by solving for $\dot x = 0$, i.e.,
$$
    \bar x = -A^{-1} B u,
$$
the result of include the duty cycle $\bar M$ required to maintain the indoor air temperature setpoint.

# Example